<a href="https://colab.research.google.com/github/Pranav-JJ/Transformers-Abstractive-Summarisation/blob/main/AbstractiveSummarisationLaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.20.0
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub
!pip install nltk
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 113.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 92.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1

In [ ]:
import os
import logging

import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# The percentage of the dataset you want to split as train and test
TRAIN_TEST_SPLIT = 0.5

MAX_INPUT_LENGTH = 512 # Maximum length of the input to the model
MIN_TARGET_LENGTH = 5  # Minimum length of the output by the model
MAX_TARGET_LENGTH = 128  # Maximum length of the output by the model
BATCH_SIZE = 2  # Batch-size for training our model
LEARNING_RATE =  1e-5  # Learning-rate for training our model
MAX_EPOCHS = 1  # Maximum number of epochs we will train the model for

# This notebook is built on the t5-small checkpoint from the Hugging Face Model Hub
MODEL_CHECKPOINT = "t5-small"

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("lighteval/legal_summarization",'BillSum', split="train")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset legal_summarization downloaded and prepared to /root/.cache/huggingface/datasets/lighteval___legal_summarization/BillSum/1.0.0/954313c09b17a47d373856977b2f8a2f26d976f28ded692bb28f2ec4bac2f84c. Subsequent calls will reuse this data.


In [ ]:
print(raw_datasets)

Dataset({
    features: ['article', 'summary'],
    num_rows: 18949
})


In [ ]:
print(raw_datasets[0])

{'article': "SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES TO NONPROFIT ORGANIZATIONS. (a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the I

In [ ]:
raw_datasets = raw_datasets.train_test_split(
    train_size=TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 9474
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 9475
    })
})


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
if MODEL_CHECKPOINT in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/9474 [00:00<?, ? examples/s]

Map:   0%|          | 0/9475 [00:00<?, ? examples/s]

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
# import keras_nlp

# rouge_l = keras_nlp.metrics.RougeL()


# def metric_fn(eval_predictions):
#     predictions, labels = eval_predictions
#     decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     for label in labels:
#         label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     result = rouge_l(decoded_labels, decoded_predictions)
#     # We will print only the F1 score, you can use other aggregation metrics as well
#     result = {"RougeL": result["f1_score"]}

#     return result

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge_scores = scorer.score(decoded_predictions, decoded_labels)

    result = {
        "Rouge1": rouge_scores['rouge1'].fmeasure,
        "Rouge2": rouge_scores['rouge2'].fmeasure,
        "RougeL": rouge_scores['rougeL'].fmeasure,
    }

    return result


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn,
    eval_dataset=generation_dataset,
    predict_with_generate=True
)


callbacks = [metric_callback]

# For now we will use our test set as our validation_data
model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)

4737/4737 [==============================] - ETA: 0s - loss: 2.6075

InvalidArgumentError: ignored

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summarizer(
    raw_datasets["test"][0]["article"],
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (1496 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Internet Radio Equality Act of 2007 - Amends the March 2, 2007, Determination of Rates and Terms of the United States Copyright Royalty Judges regarding rates and terms for the digital performance of sound recordings and ephemeral recordings, including that determination as modified by the April 17, 2007, Order Denying Motions for Rehearing and any subsequent modification to that determination by the Copyright royalty judges that is published in the Federal Register, are not effective, and shall be deemed never to have been effective.'}]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
output_dir = "model"  # Replace with your desired output directory

# Save the model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/tokenizer.json')

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="T5-Finetuned-legal_summarization",
    tokenizer=tokenizer,
)


Cloning https://huggingface.co/phoen1x/T5-Finetuned-legal_summarization into local empty directory.


In [ ]:
model.push_to_hub("T5-Finetuned-legal_summarization", organization="keras-io")
tokenizer.push_to_hub("T5-Finetuned-legal_summarization", organization="keras-io")

Upload file tf_model.h5:   0%|          | 1.00/231M [00:00<?, ?B/s]

To https://huggingface.co/phoen1x/T5-Finetuned-legal_summarization
   e6aed35..e32441b  main -> main

   e6aed35..e32441b  main -> main

To https://huggingface.co/phoen1x/T5-Finetuned-legal_summarization
   e32441b..a1e41b2  main -> main

   e32441b..a1e41b2  main -> main



'https://huggingface.co/phoen1x/T5-Finetuned-legal_summarization/commit/a1e41b22b168f687174aab21ca4b227da4c6bb34'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("phoen1x/T5-Finetuned-legal_summarization")

model = AutoModelForSeq2SeqLM.from_pretrained("phoen1x/T5-Finetuned-legal_summarization", from_tf=True)

Downloading:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


In [ ]:
text="summarize: SECTION 1. LIABILITY OF BUSINESS ENTITIES PROVIDING USE OF FACILITIES TO NONPROFIT ORGANIZATIONS. (a) Definitions.--In this section: (1) Business entity.--The term ``business entity'' means a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise. (2) Facility.--The term ``facility'' means any real property, including any building, improvement, or appurtenance. (3) Gross negligence.--The term ``gross negligence'' means voluntary and conscious conduct by a person with knowledge (at the time of the conduct) that the conduct is likely to be harmful to the health or well-being of another person. (4) Intentional misconduct.--The term ``intentional misconduct'' means conduct by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person. (5) Nonprofit organization.--The term ``nonprofit organization'' means-- (A) any organization described in section 501(c)(3) of the Internal Revenue Code of 1986 and exempt from tax under section 501(a) of such Code; or (B) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes. (6) State.--The term ``State'' means each of the several States, the District of Columbia, the Commonwealth of Puerto Rico, the Virgin Islands, Guam, American Samoa, the Northern Mariana Islands, any other territory or possession of the United States, or any political subdivision of any such State, territory, or possession. (b) Limitation on Liability.-- (1) In general.--Subject to subsection (c), a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization if-- (A) the use occurs outside of the scope of business of the business entity; (B) such injury or death occurs during a period that such facility is used by the nonprofit organization; and (C) the business entity authorized the use of such facility by the nonprofit organization. (2) Application.--This subsection shall apply-- (A) with respect to civil liability under Federal and State law; and (B) regardless of whether a nonprofit organization pays for the use of a facility. (c) Exception for Liability.--Subsection (b) shall not apply to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including any misconduct that-- (1) constitutes a crime of violence (as that term is defined in section 16 of title 18, United States Code) or act of international terrorism (as that term is defined in section 2331 of title 18) for which the defendant has been convicted in any court; (2) constitutes a hate crime (as that term is used in the Hate Crime Statistics Act (28 U.S.C. 534 note)); (3) involves a sexual offense, as defined by applicable State law, for which the defendant has been convicted in any court; or (4) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law. (d) Superseding Provision.-- (1) In general.--Subject to paragraph (2) and subsection (e), this Act preempts the laws of any State to the extent that such laws are inconsistent with this Act, except that this Act shall not preempt any State law that provides additional protection from liability for a business entity for an injury or death with respect to which conditions under subparagraphs (A) through (C) of subsection (b)(1) apply. (2) Limitation.--Nothing in this Act shall be construed to supersede any Federal or State health or safety law. (e) Election of State Regarding Nonapplicability.--This Act shall not apply to any civil action in a State court against a business entity in which all parties are citizens of the State if such State enacts a statute-- (1) citing the authority of this subsection; (2) declaring the election of such State that this Act shall not apply to such civil action in the State; and (3) containing no other provision."


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="phoen1x/T5-Finetuned-legal_summarization")
summarizer(text)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at phoen1x/T5-Finetuned-legal_summarization.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (987 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Business entity: (1) a firm, corporation, association, partnership, consortium, joint venture, or other form of enterprise; (2) any real property, including any building, improvement, or appurtenance; and (3) gross negligence by a person with knowledge (at the time of the conduct) that the conduct is harmful to the health or well-being of another person; and (2) any not-for-profit organization organized and conducted for public benefit and operated primarily for charitable, civic, educational, religious, welfare, or health purposes.'}]

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

reference_summary = "Shields a business entity from civil liability relating to any injury or death occurring at a facility of that entity in connection with a use of such facility by a nonprofit organization if: (1) the use occurs outside the scope of business of the business entity; (2) such injury or death occurs during a period that such facility is used by such organization; and (3) the business entity authorized the use of such facility by the organization. Makes this Act inapplicable to an injury or death that results from an act or omission of a business entity that constitutes gross negligence or intentional misconduct, including misconduct that: (1) constitutes a hate crime or a crime of violence or act of international terrorism for which the defendant has been convicted in any court; or (2) involves a sexual offense for which the defendant has been convicted in any court or misconduct for which the defendant has been found to have violated a Federal or State civil rights law. Preempts State laws to the extent that such laws are inconsistent with this Act, except State law that provides additional protection from liability. Specifies that this Act shall not be construed to supersede any Federal or State health or safety law. Makes this Act inapplicable to any civil action in a State court against a business entity in which all parties are citizens of the State if such State, citing this Act's authority and containing no other provision, enacts a statute declaring the State's election that this Act shall not apply to such action in the State."

generated_summary = "Exception for Liability: (1) a business entity shall not be subject to civil liability relating to any injury or death occurring at a facility of the business entity in connection with a use of such facility by a nonprofit organization; (2) constitutes a crime of violence or act of international terrorism; and (3) involves misconduct for which the defendant has been found to have violated a Federal or State civil rights law."


scores = scorer.score(reference_summary, generated_summary)

print(scores['rouge1'].fmeasure)  # Print F1 score for ROUGE-1
print(scores['rouge2'].fmeasure)  # Print F1 score for ROUGE-2
print(scores['rougeL'].fmeasure)  # Print F1 score for ROUGE-L



0.41420118343195267
0.3452380952380953
0.34911242603550297
